In [1]:
"""# Install all required packages
!pip install torch torchvision transformers
!pip install pillow requests
!pip install gtts pygame
!pip install SpeechRecognition
!pip install opencv-python
!pip install accelerate
!pip install pydub  # For audio file handling"""
import torch
import torchvision.transforms as transforms
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import requests
import io
import base64
import json
import time
import threading
import queue
import os
import tempfile
from datetime import datetime

# TTS and ASR imports
from gtts import gTTS
import pygame
import speech_recognition as sr

# Display and utility imports
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Audio, clear_output
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages installed and imported successfully!")


pygame 2.6.1 (SDL 2.28.4, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
✅ All packages installed and imported successfully!


In [2]:
class AgricultureVLM:
    def __init__(self, model_name="vikhyatk/moondream2"):
        """
        Initialize the agriculture-focused Vision Language Model
        Using Moondream2 - a small (2B) but capable VLM perfect for cloud environments
        """
        print("🚀 Loading Vision Language Model...")
        
        # Device setup
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🖥️  Using device: {self.device}")
        
        try:
            # Load Moondream2 model
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=True,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto" if self.device == "cuda" else None
            ).to(self.device)
            
            self.tokenizer = AutoTokenizer.from_pretrained(
                model_name, 
                trust_remote_code=True
            )
            
            # Agriculture-specific system prompt
            self.agriculture_context = """
            You are an expert agricultural assistant with deep knowledge in:
            - Crop health and disease identification
            - Pest detection and integrated pest management
            - Soil conditions and nutrient management
            - Irrigation and water management strategies
            - Harvest timing and post-harvest techniques
            - Weather impact assessment and mitigation
            - Sustainable and organic farming practices
            - Crop rotation and companion planting
            
            Always provide practical, actionable advice suitable for farmers.
            Focus on cost-effective and locally appropriate solutions.
            """
            
            print("✅ Vision Language Model loaded successfully!")
            
        except Exception as e:
            print(f"❌ Error loading VLM: {str(e)}")
            raise e
    
    def process_query(self, image, text_query):
        """
        Process image and text query for agriculture-specific insights
        """
        try:
            # Prepare the enhanced prompt
            enhanced_prompt = f"""
            {self.agriculture_context}
            
            Farmer's Question: {text_query}
            
            Please analyze the image and provide specific, actionable advice:
            """
            
            # Generate response using the VLM
            response = self.model.answer_question(image, enhanced_prompt, self.tokenizer)
            
            return response
            
        except Exception as e:
            error_msg = f"Error processing agricultural query: {str(e)}"
            print(f"❌ {error_msg}")
            return error_msg

# Initialize the VLM
print("🌾 Initializing Agriculture VLM...")
vlm_assistant = AgricultureVLM()


🌾 Initializing Agriculture VLM...
🚀 Loading Vision Language Model...
🖥️  Using device: cuda


✅ Vision Language Model loaded successfully!


In [3]:
class AgricultureTTS:
    def __init__(self, language='en', output_dir='audio_responses'):
        """
        Enhanced TTS for agriculture responses with file saving for cloud environments
        """
        self.language = language
        self.output_dir = output_dir
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Initialize pygame for local audio playback (optional)
        try:
            pygame.mixer.init()
            self.pygame_available = True
            print("🔊 Audio playback available")
        except:
            self.pygame_available = False
            print("⚠️  Audio playback not available (cloud environment)")
    
    def generate_speech_file(self, text, filename=None, slow=False):
        """
        Generate speech file from text - works in all environments
        """
        try:
            if filename is None:
                timestamp = int(time.time())
                filename = f"agriculture_response_{timestamp}.mp3"
            
            # Ensure filename has .mp3 extension
            if not filename.endswith('.mp3'):
                filename += '.mp3'
            
            filepath = os.path.join(self.output_dir, filename)
            
            # Create TTS object and save
            tts = gTTS(text=text, lang=self.language, slow=slow)
            tts.save(filepath)
            
            print(f"💾 Speech saved: {filepath}")
            
            # Try to play if possible (works locally)
            if self.pygame_available:
                try:
                    pygame.mixer.music.load(filepath)
                    pygame.mixer.music.play()
                    
                    # Wait for playback to finish
                    while pygame.mixer.music.get_busy():
                        time.sleep(0.1)
                    
                    print("🔊 Audio played successfully")
                except:
                    print("🔇 Audio playback failed (normal in cloud environments)")
            
            return filepath
            
        except Exception as e:
            print(f"❌ TTS Error: {str(e)}")
            return None
    
    def speak_response(self, text, save_file=True, filename=None):
        """
        Generate speech response (with optional file saving)
        """
        if save_file:
            return self.generate_speech_file(text, filename)
        else:
            # Quick playback without saving (if possible)
            try:
                tts = gTTS(text=text, lang=self.language)
                with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp_file:
                    tts.save(tmp_file.name)
                    
                    if self.pygame_available:
                        pygame.mixer.music.load(tmp_file.name)
                        pygame.mixer.music.play()
                        
                        while pygame.mixer.music.get_busy():
                            time.sleep(0.1)
                    
                    os.unlink(tmp_file.name)
                    return True
            except Exception as e:
                print(f"TTS Error: {e}")
                return False

# Initialize TTS
print("🎵 Initializing Text-to-Speech...")
tts_system = AgricultureTTS()


🎵 Initializing Text-to-Speech...
⚠️  Audio playback not available (cloud environment)


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


In [8]:

import speech_recognition as sr
from pydub import AudioSegment
import tempfile
import os

class AgricultureASR:
    def __init__(self, supported_formats=None):
        """
        Enhanced ASR with proper MP3 support via conversion
        """
        self.recognizer = sr.Recognizer()
        self.supported_formats = supported_formats or ['.wav', '.mp3', '.flac', '.m4a', '.aiff']
        
        print("🎤 Enhanced Speech Recognition initialized")
        print(f"📁 Supported formats: {', '.join(self.supported_formats)}")
    
    def convert_to_wav(self, audio_file_path):
        """
        Convert any audio format to WAV for speech recognition
        """
        try:
            file_extension = os.path.splitext(audio_file_path)[1].lower()
            
            if file_extension == '.wav':
                return audio_file_path  # Already WAV
            
            print(f"🔄 Converting {file_extension} to WAV...")
            
            # Load audio file with pydub
            if file_extension == '.mp3':
                audio = AudioSegment.from_mp3(audio_file_path)
            elif file_extension == '.m4a':
                audio = AudioSegment.from_file(audio_file_path, format="m4a")
            elif file_extension == '.flac':
                audio = AudioSegment.from_file(audio_file_path, format="flac")
            else:
                audio = AudioSegment.from_file(audio_file_path)
            
            # Create temporary WAV file
            temp_wav = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
            
            # Export as WAV with proper settings for speech recognition
            audio.export(
                temp_wav.name,
                format="wav",
                parameters=["-ac", "1", "-ar", "16000"]  # Mono, 16kHz
            )
            
            print(f"✅ Converted to: {temp_wav.name}")
            return temp_wav.name
            
        except Exception as e:
            print(f"❌ Conversion error: {str(e)}")
            return None
    
    def validate_audio_file(self, audio_file_path):
        """
        Validate audio file
        """
        if not os.path.exists(audio_file_path):
            print(f"❌ Audio file not found: {audio_file_path}")
            return False
        
        file_extension = os.path.splitext(audio_file_path)[1].lower()
        if file_extension not in self.supported_formats:
            print(f"❌ Unsupported format: {file_extension}")
            print(f"Supported formats: {', '.join(self.supported_formats)}")
            return False
        
        return True
    
    def transcribe_audio_file(self, audio_file_path, language='en-US'):
        """
        Convert audio file to text with proper MP3 support
        """
        converted_file = None
        try:
            # Validate file first
            if not self.validate_audio_file(audio_file_path):
                return None
            
            print(f"🎵 Processing audio file: {os.path.basename(audio_file_path)}")
            
            # Convert to WAV if needed
            wav_file = self.convert_to_wav(audio_file_path)
            if wav_file is None:
                return None
            
            converted_file = wav_file if wav_file != audio_file_path else None
            
            # Process with speech recognition
            with sr.AudioFile(wav_file) as source:
                self.recognizer.adjust_for_ambient_noise(source, duration=0.5)
                audio_data = self.recognizer.record(source)
            
            print("🔄 Converting speech to text...")
            
            # Recognize speech
            text = self.recognizer.recognize_google(audio_data, language=language)
            print(f"✅ Transcribed: '{text}'")
            
            return text
            
        except sr.UnknownValueError:
            print("❌ Could not understand the audio in the file")
            return None
        except sr.RequestError as e:
            print(f"❌ ASR service error: {e}")
            return None
        except Exception as e:
            print(f"❌ Error processing audio file: {e}")
            return None
        finally:
            # Clean up temporary WAV file
            if converted_file and os.path.exists(converted_file):
                try:
                    os.unlink(converted_file)
                    print("🗑️  Temporary file cleaned up")
                except:
                    pass

asr_system = AgricultureASR()

🎤 Enhanced Speech Recognition initialized
📁 Supported formats: .wav, .mp3, .flac, .m4a, .aiff


In [9]:
class AgricultureAssistant:
    def __init__(self, vlm, tts, asr):
        """
        Complete agriculture assistant with VLM, TTS, and file-based ASR
        """
        self.vlm = vlm
        self.tts = tts
        self.asr = asr
        self.conversation_history = []
        self.current_image = None
        self.session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        print("🌾 Agriculture Assistant initialized successfully!")
        print(f"📊 Session ID: {self.session_id}")
    
    def load_image(self, image_path_or_url):
        """
        Load image from file path or URL
        """
        try:
            print(f"📷 Loading image: {image_path_or_url}")
            
            if image_path_or_url.startswith(('http://', 'https://')):
                response = requests.get(image_path_or_url)
                response.raise_for_status()
                image = Image.open(io.BytesIO(response.content))
                print("✅ Image loaded from URL")
            else:
                image = Image.open(image_path_or_url)
                print("✅ Image loaded from file")
            
            # Convert to RGB if necessary
            if image.mode != 'RGB':
                image = image.convert('RGB')
            
            self.current_image = image
            
            # Display basic image info
            print(f"📐 Image size: {image.size}")
            print(f"🎨 Image mode: {image.mode}")
            
            return image
            
        except Exception as e:
            error_msg = f"❌ Error loading image: {str(e)}"
            print(error_msg)
            return None
    
    def display_image(self, figsize=(10, 8)):
        """
        Display the current loaded image
        """
        if self.current_image is None:
            print("❌ No image loaded. Use load_image() first.")
            return
        
        plt.figure(figsize=figsize)
        plt.imshow(self.current_image)
        plt.axis('off')
        plt.title("🌾 Agriculture Image for Analysis", fontsize=14, pad=20)
        plt.tight_layout()
        plt.show()
    
    def process_text_query(self, text_query, generate_voice_response=True):
        """
        Process text query with current image
        """
        if self.current_image is None:
            response = "❌ Please load an image first using load_image() method."
            print(response)
            return response, None
        
        print(f"🌾 Processing query: '{text_query}'")
        print("🔄 Analyzing with VLM...")
        
        # Get VLM response
        response = self.vlm.process_query(self.current_image, text_query)
        
        # Generate unique response filename
        response_filename = f"response_{self.session_id}_{len(self.conversation_history)+1}.mp3"
        
        # Add to conversation history
        conversation_entry = {
            'type': 'text_query',
            'query': text_query,
            'response': response,
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'session_id': self.session_id
        }
        
        # Generate voice response if requested
        audio_file = None
        if generate_voice_response:
            print("🔊 Generating voice response...")
            audio_file = self.tts.generate_speech_file(response, response_filename)
            conversation_entry['response_audio'] = audio_file
        
        self.conversation_history.append(conversation_entry)
        
        print(f"\n📝 Response: {response}\n")
        if audio_file:
            print(f"🎵 Voice response saved: {audio_file}")
        
        return response, audio_file
    
    def process_voice_query(self, audio_file_path, generate_voice_response=True, language='en-US'):
        """
        Process voice query from audio file
        """
        if self.current_image is None:
            response = "❌ Please load an image first using load_image() method."
            print(response)
            if generate_voice_response:
                audio_file = self.tts.generate_speech_file(response, "error_no_image.mp3")
                return response, None, audio_file
            return response, None, None
        
        print(f"🎤 Processing voice query from: {os.path.basename(audio_file_path)}")
        
        # Get audio file info
        self.asr.get_audio_info(audio_file_path)
        
        # Transcribe audio to text
        text_query = self.asr.transcribe_audio_file(audio_file_path, language)
        
        if text_query is None:
            response = "❌ Could not understand the audio. Please ensure clear speech and supported audio format."
            print(response)
            if generate_voice_response:
                audio_file = self.tts.generate_speech_file(response, "error_transcription.mp3")
                return response, None, audio_file
            return response, None, None
        
        print(f"🌾 Transcribed query: '{text_query}'")
        print("🔄 Analyzing with VLM...")
        
        # Get VLM response
        vlm_response = self.vlm.process_query(self.current_image, text_query)
        
        # Generate unique response filename
        response_filename = f"voice_response_{self.session_id}_{len(self.conversation_history)+1}.mp3"
        
        # Add to conversation history
        conversation_entry = {
            'type': 'voice_query',
            'audio_file': audio_file_path,
            'transcribed_query': text_query,
            'response': vlm_response,
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'session_id': self.session_id
        }
        
        # Generate voice response if requested
        response_audio_file = None
        if generate_voice_response:
            print("🔊 Generating voice response...")
            response_audio_file = self.tts.generate_speech_file(vlm_response, response_filename)
            conversation_entry['response_audio'] = response_audio_file
        
        self.conversation_history.append(conversation_entry)
        
        print(f"\n📝 Response: {vlm_response}\n")
        if response_audio_file:
            print(f"🎵 Voice response saved: {response_audio_file}")
        
        return vlm_response, text_query, response_audio_file
    
    def batch_voice_analysis(self, image_path, audio_files_list, language='en-US'):
        """
        Analyze multiple voice queries for the same image
        """
        # Load the image
        if not self.load_image(image_path):
            return None
        
        self.display_image()
        
        print(f"🔄 Starting batch analysis of {len(audio_files_list)} voice queries...")
        print("="*80)
        
        batch_results = []
        
        for i, audio_file in enumerate(audio_files_list, 1):
            print(f"\n🎤 Processing query {i}/{len(audio_files_list)}: {os.path.basename(audio_file)}")
            print("-" * 60)
            
            response, query, response_audio = self.process_voice_query(
                audio_file, 
                generate_voice_response=True,
                language=language
            )
            
            result = {
                'query_number': i,
                'audio_file': audio_file,
                'transcribed_query': query,
                'vlm_response': response,
                'response_audio_file': response_audio,
                'processing_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            
            batch_results.append(result)
            
            print(f"✅ Query {i} completed")
            time.sleep(2)  # Brief pause between queries
        
        print("="*80)
        print(f"🎉 Batch analysis completed! Processed {len(batch_results)} queries.")
        
        return batch_results
    
    def show_conversation_history(self, save_to_file=False):
        """
        Display and optionally save conversation history
        """
        if not self.conversation_history:
            print("📭 No conversation history yet.")
            return
        
        print(f"\n📚 Conversation History (Session: {self.session_id})")
        print("=" * 80)
        
        history_text = []
        
        for i, entry in enumerate(self.conversation_history, 1):
            print(f"\n{i}. [{entry['timestamp']}] - {entry['type'].title()}")
            
            if entry['type'] == 'voice_query':
                print(f"   🎤 Audio File: {os.path.basename(entry['audio_file'])}")
                print(f"   📝 Transcribed: {entry['transcribed_query']}")
                if 'response_audio' in entry:
                    print(f"   🔊 Response Audio: {os.path.basename(entry['response_audio'])}")
            else:
                print(f"   📝 Query: {entry['query']}")
            
            print(f"   🤖 Response: {entry['response'][:100]}...")
            
            # Add to history text for saving
            history_text.append(f"{i}. [{entry['timestamp']}] - {entry['type'].title()}")
            if entry['type'] == 'voice_query':
                history_text.append(f"   Audio: {entry['audio_file']}")
                history_text.append(f"   Query: {entry['transcribed_query']}")
            else:
                history_text.append(f"   Query: {entry['query']}")
            history_text.append(f"   Response: {entry['response']}")
            history_text.append("")
        
        print("=" * 80)
        
        # Save to file if requested
        if save_to_file:
            history_filename = f"conversation_history_{self.session_id}.txt"
            with open(history_filename, 'w', encoding='utf-8') as f:
                f.write(f"Agriculture Assistant Conversation History\n")
                f.write(f"Session ID: {self.session_id}\n")
                f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("=" * 80 + "\n\n")
                f.write("\n".join(history_text))
            
            print(f"💾 History saved to: {history_filename}")
    
    def get_session_summary(self):
        """
        Get summary of current session
        """
        total_queries = len(self.conversation_history)
        voice_queries = len([h for h in self.conversation_history if h['type'] == 'voice_query'])
        text_queries = total_queries - voice_queries
        
        summary = {
            'session_id': self.session_id,
            'total_queries': total_queries,
            'voice_queries': voice_queries,
            'text_queries': text_queries,
            'has_image_loaded': self.current_image is not None,
            'session_start': self.conversation_history[0]['timestamp'] if self.conversation_history else None,
            'last_activity': self.conversation_history[-1]['timestamp'] if self.conversation_history else None
        }
        
        return summary

# Initialize the complete agriculture assistant
print("🚀 Initializing Complete Agriculture Assistant...")
agriculture_assistant = AgricultureAssistant(vlm_assistant, tts_system, asr_system)


🚀 Initializing Complete Agriculture Assistant...
🌾 Agriculture Assistant initialized successfully!
📊 Session ID: 20250808_055518


In [17]:

AGRICULTURE_QUERIES = {
    "disease_detection": "Analyze this crop image for any plant diseases. What symptoms do you see and what treatment do you recommend?",
    "pest_identification": "Identify any pests or insects visible in this image. What damage are they causing and how should I control them?",
    "nutrient_deficiency": "Examine the plant leaves and overall appearance. Do you see signs of nutrient deficiency? What fertilizers should I apply?",
    "growth_stage": "What growth stage is this crop in? What specific care and management does it need at this stage?",
    "harvest_readiness": "Is this crop ready for harvest? What indicators should I look for to determine optimal harvest timing?",
    "soil_health": "Assess the soil condition visible in this image. What improvements can I make for better crop production?",
    "irrigation_needs": "Based on the plant and soil condition, what are the current irrigation requirements?",
    "weather_damage": "Do you see any weather-related damage to these crops? How can I protect them better in the future?"
}

def demo_single_voice_query():
    """
    Demo: Single voice query analysis
    """
    print("🎤 Demo: Single Voice Query Analysis")
    print("-" * 50)
    
    # Example usage (replace with your actual files)
    image_path = "crop_image.jpg"  # Replace with your crop image
    audio_file = "farmer_question.wav"  # Replace with your audio file
    
    print("📋 Steps to use:")
    print("1. Replace 'crop_image.jpg' with your actual crop image path")
    print("2. Replace 'farmer_question.wav' with your actual audio file path")
    print("3. Run the analysis")
    
    # Example code structure:
    example_code = '''
    # Load crop image
    agriculture_assistant.load_image("your_crop_image.jpg")
    agriculture_assistant.display_image()
    
    # Process voice query
    response, query, audio_response = agriculture_assistant.process_voice_query(
        "your_audio_question.wav",
        generate_voice_response=True
    )
    '''
    print(f"💻 Example code:\n{example_code}")

def demo_batch_analysis():
    """
    Demo: Batch voice query analysis
    """
    print("🔄 Demo: Batch Voice Analysis")
    print("-" * 50)
    
    example_code = '''
    # Multiple audio files for the same crop
    audio_files = [
        "question1.wav",  # "What disease is affecting my tomatoes?"
        "question2.wav",  # "How should I treat this plant?"
        "question3.wav"   # "When should I apply fertilizer?"
    ]
    
    # Run batch analysis
    results = agriculture_assistant.batch_voice_analysis(
        image_path="tomato_crop.jpg",
        audio_files_list=audio_files
    )
    '''
    print(f"💻 Example code:\n{example_code}")

def demo_quick_analysis(query_type="disease_detection"):
    """
    Demo: Quick analysis with predefined queries
    """
    if query_type in AGRICULTURE_QUERIES:
        query = AGRICULTURE_QUERIES[query_type]
        print(f"🎯 Quick Analysis: {query_type.replace('_', ' ').title()}")
        print(f"📝 Query: {query}")
        
        example_code = f'''
        # Load image
        agriculture_assistant.load_image("your_crop_image.jpg")
        
        # Run quick analysis
        response, audio_file = agriculture_assistant.process_text_query(
            "{query}",
            generate_voice_response=True
        )
        '''
        print(f"💻 Code:\n{example_code}")
    else:
        print(f"Available analysis types: {list(AGRICULTURE_QUERIES.keys())}")

def create_recording_guide():
    """
    Guide for recording audio queries
    """
    print("🎙️ Audio Recording Guide for Farmers")
    print("=" * 60)
    
    recording_tips = [
        "🔇 Record in a quiet environment",
        "🗣️  Speak clearly and at normal speed", 
        "⏱️  Keep recordings between 5-30 seconds",
        "📱 Use your phone's voice recorder app",
        "💾 Save as WAV, MP3, or FLAC format",
        "🌐 Ensure internet connection for transcription"
    ]
    
    sample_queries = [
        "What disease is affecting my crops?",
        "How should I treat these brown spots on leaves?",
        "Is this crop ready for harvest?",
        "What fertilizer should I apply now?",
        "How can I control these pests?",
        "What is causing the yellowing of leaves?"
    ]
    
    print("📋 Recording Tips:")
    for tip in recording_tips:
        print(f"  {tip}")
    
    print("\n🎤 Sample Questions to Record:")
    for i, query in enumerate(sample_queries, 1):
        print(f"  {i}. \"{query}\"")
    
    print("\n📁 Supported Audio Formats:")
    for fmt in ['.wav', '.mp3', '.flac', '.m4a']:
        print(f"  ✅ {fmt}")

# Run demos
print("🎉 Agriculture VLM Assistant Ready!")
print("\n🚀 Available Demo Functions:")
print("1. demo_single_voice_query()")
print("2. demo_batch_analysis()")  
print("3. demo_quick_analysis('disease_detection')")
print("4. create_recording_guide()")

print(f"\n🎯 Available Analysis Types:")
for key, desc in AGRICULTURE_QUERIES.items():
    print(f"  • {key}: {desc[:50]}...")

print(f"\n📊 Session Info:")
summary = agriculture_assistant.get_session_summary()
for key, value in summary.items():
    print(f"  {key}: {value}")


🎉 Agriculture VLM Assistant Ready!

🚀 Available Demo Functions:
1. demo_single_voice_query()
2. demo_batch_analysis()
3. demo_quick_analysis('disease_detection')
4. create_recording_guide()

🎯 Available Analysis Types:
  • disease_detection: Analyze this crop image for any plant diseases. Wh...
  • pest_identification: Identify any pests or insects visible in this imag...
  • nutrient_deficiency: Examine the plant leaves and overall appearance. D...
  • growth_stage: What growth stage is this crop in? What specific c...
  • harvest_readiness: Is this crop ready for harvest? What indicators sh...
  • soil_health: Assess the soil condition visible in this image. W...
  • irrigation_needs: Based on the plant and soil condition, what are th...
  • weather_damage: Do you see any weather-related damage to these cro...

📊 Session Info:
  session_id: 20250806_051248
  total_queries: 0
  voice_queries: 0
  text_queries: 0
  has_image_loaded: False
  session_start: None
  last_activity: None
